In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [ ]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+ 'sakila'
engine = create_engine(connection_string)
engine

In [ ]:
sqlalchemy.create_engine(connection_string)

In [ ]:
connection = engine.connect()

In [ ]:
from sqlalchemy import text

with engine.connect() as connection:
    query = text('SELECT * FROM rental')
    result = connection.execute(query)
    print(pd.DataFrame(result.all()))


In [ ]:
def rental_month (engine, year, month):
    with engine.connect() as connection:
        txt = '''SELECT * FROM rental 
        WHERE YEAR(rental_date) = :year
        AND MONTH(rental_date) = :month;'''
        query = text(txt)
        result = connection.execute(query, {"year": year, "month": month})
        df =  pd.DataFrame(result.fetchall(), columns=result.keys())
    return df


rental_month(engine, 2005, 5)

In [ ]:
def rental_count_month(df):
    rental_counts = df.groupby('customer_id').size().reset_index(name='rental_count')
    return rental_counts

rental_count_month(df)

# Get rental data for May 2005
rental_data = rental_month(engine, 2005, 5)

# Compute rental counts for each customer
rental_counts = rental_count_month(rental_data)
print(rental_counts.head())

In [ ]:
def compare_rentals(df1, df2):
    df1 = df1.rename(columns={'rental_count': 'rental_count_1'})
    df2 = df2.rename(columns={'rental_count': 'rental_count_2'})
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer')
    combined_df.fillna(0, inplace=True)
    combined_df['difference'] = combined_df['rental_count_1'] - combined_df['rental_count_2']
    return combined_df

# Get rental data for May 2005
rental_data1 = rental_month(engine, 2005, 5)
rental_counts1 = rental_count_month(rental_data1)

# Get rental data for June 2005
rental_data2 = rental_month(engine, 2005, 6)
rental_counts2 = rental_count_month(rental_data2)

# Compare the rental counts between May and June 2005
comparison = compare_rentals(rental_counts1, rental_counts2)

# Display the comparison DataFrame
print(comparison.head())